In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time
import random
from lightgbm.sklearn import LGBMRegressor
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
# For preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
# For LSTM model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from tqdm.keras import TqdmCallback
# For FOPDT model
from scipy.integrate import odeint
# For hyperopt (parameter optimization)
from hyperopt import Trials, STATUS_OK, tpe, fmin, hp
from tensorflow.keras.optimizers import Adadelta, Adam, RMSprop,Adagrad,Adadelta,Adamax,Nadam,SGD

import matplotlib.image as mpimg
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.utils import shuffle
from itertools import cycle
import keras
import seaborn as sns
from sklearn.model_selection import KFold
import pandas as pd
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from tensorflow.keras.layers import BatchNormalization
import os
from skimage import color

# Import libraries and packages
import matplotlib as plt
plt.style.use('ggplot')
%matplotlib inline
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.layers.core import Dense
from tensorflow.keras.optimizers import SGD
from imutils import paths
import random
import pickle
import cv2
import os
import time   # time1 = time.time(); print('Time taken: {:.1f} seconds'.format(time.time() - time1))

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.datasets import make_classification
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.preprocessing import MultiLabelBinarizer
SEED = 42   # set random seed

In [ ]:
#โหลดดาต้า
X = np.load(r'C:\Users\ASUS\Documents\datasci\dataset\detect\Sample_HOG_embreded.npy') 
y = np.load(r'C:\Users\ASUS\Documents\datasci\dataset\detect\Sample_labels_HOG.npy')

In [ ]:
X.shape

In [ ]:
#เเปลงเป็น binary
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y2 = label_encoder.fit_transform(y)

In [ ]:
num_classes = 8
Y = np_utils.to_categorical(y2, num_classes)

#Shuffle the dataset
X,y = shuffle(X,Y, random_state=42)

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.2,shuffle=False)   

In [ ]:
#ใส่ค่า parameter ที่ต้องการหา 
from hyperopt.pyll.base import scope
space = { 
            'layers': scope.int(hp.quniform('layers',1,3,1)),
            'units1': scope.int(hp.quniform('units1',10,200,5)),
            'units2': scope.int(hp.quniform('units2',10,200,5)),
            'units3': scope.int(hp.quniform('units3',10,200,5)),

            'dropout1': hp.quniform('dropout1',0.01,0.5,0.01),
            'dropout2': hp.quniform('dropout2',0.01,0.5,0.01),
            'dropout3': hp.quniform('dropout3',0.01,0.5,0.01),

            'kernel1' : hp.choice('kernel1',['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']),
            'kernel2' : hp.choice('kernel2',['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']),
            'kernel3' : hp.choice('kernel3',['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']),

            'activation1': hp.choice('activation1',['softmax','softsign','relu','elu','tanh','swish','sigmoid','leaky_relu']),
            'activation2': hp.choice('activation2',['softmax','softsign','relu','elu','tanh','swish','sigmoid','leaky_relu']),
            'activation3': hp.choice('activation3',['softmax','softsign','relu','elu','tanh','swish','sigmoid','leaky_relu']),
    
            'optimizer': hp.choice('optimizer1',['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']),
            'batch_size' : scope.int(hp.quniform('batch_size',5,60,5)),
            'epochs' : scope.int(hp.quniform('epochs',10,300,5)),
            
        }


In [ ]:
def f_nn(params):
    # Generate data with given window
    
    model = Sequential()
    model=keras.Sequential()
    if params['layers'] == 1:
        model.add(Dense(units=int(params['units1']),kernel_initializer=params['kernel1'],
                                    input_shape=(7182,),
                                    kernel_regularizer=keras.regularizers.l2(0.01)))
        model.add(Dropout(params['dropout1']))
        model.add(Activation(params['activation1']))
        
        
    elif params['layers'] == 2:
        model.add(Dense(units=int(params['units1']),kernel_initializer=params['kernel1'],
                                    input_shape=(7182,),
                                    kernel_regularizer=keras.regularizers.l2(0.01)))
        model.add(Dropout(params['dropout1']))
        model.add(Activation(params['activation1']))
        ##layer2
        model.add(Dense(units=int(params['units2']),kernel_initializer=params['kernel2']))
        model.add(Dropout(params['dropout2']))
        model.add(Activation(params['activation2']))
        
        
    elif params['layers'] == 3:
        model.add(Dense(units=int(params['units1']),kernel_initializer=params['kernel1'],
                                    input_shape=(7182,),
                                    kernel_regularizer=keras.regularizers.l2(0.01)))
        model.add(Dropout(params['dropout1']))
        model.add(Activation(params['activation1']))
        ##layer2
        model.add(Dense(units=int(params['units2']),kernel_initializer=params['kernel2']))
        model.add(Dropout(params['dropout2']))
        model.add(Activation(params['activation2']))
        ##layer3
        model.add(Dense(units=int(params['units3']),kernel_initializer=params['kernel3']))
        model.add(Dropout(params['dropout3']))
        model.add(Activation(params['activation3']))
        
        
    model.add(Dense(8, activation='softmax'))
    model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer=params['optimizer'])

    es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=15)
#
    result = model.fit(Xtrain, ytrain, verbose=0, validation_split=0.1,
                       batch_size=params['batch_size'],
                       epochs=params['epochs'],
                       callbacks = [es,TqdmCallback(verbose=1)]
                      )
    
    #get the lowest validation loss of the training epochs
    validation_loss = np.amin(result.history['val_loss']) 
    print('Best validation loss of epoch:', validation_loss)

    return {'loss': validation_loss, 'status': STATUS_OK, 'model': model, 'params': params}

In [ ]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=100, trials=trials) #กำหนด max evals จำนวนครั้งวนซ้ำ

best_model = trials.results[np.argmin([r['loss'] for r in trials.results])]['model']
best_params = trials.results[np.argmin([r['loss'] for r in trials.results])]['params']

worst_model = trials.results[np.argmax([r['loss'] for r in trials.results])]['model']
worst_params = trials.results[np.argmax([r['loss'] for r in trials.results])]['params']

In [ ]:
best_model

In [ ]:
best_params

In [ ]:
worst_params

In [ ]:
best_model.get_config()

In [ ]:
best_model.summary()

In [ ]:
trials.results